In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL da página
url = "https://fbref.com/pt/comps/9/cronograma/Premier-League-Resultados-e-Calendarios"
# Faz a requisição para a página
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Função para extrair a tabela da página
def extrair_tabela(rows):
    headers = []
    data = []
    
    for row in rows:
        cols = row.find_all(['th', 'td'])
        if len(headers) == 0:  # A primeira linha será o cabeçalho
            headers = [col.get_text(strip=True) for col in cols]
        else:  # Demais linhas são dados
            data.append([col.get_text(strip=True) for col in cols])
    
    # Cria o DataFrame
    return pd.DataFrame(data, columns=headers)

# Encontra o container da tabela
table_container = soup.find('div', {'class': 'table_container'})
rows = table_container.find_all('tr')

# Extrai a tabela
tabela = extrair_tabela(rows)

# Encontra a coluna "Relatório da Partida"
links_relatorios = []
for row in rows:
    a_tag = row.find('a', text='Relatório da Partida')
    if a_tag:
        href = a_tag.get('href')
        if href and href != "#":  # Ignora links vazios ou "#" como marcador
            links_relatorios.append(href)
        else:
            links_relatorios.append(None)
    else:
        links_relatorios.append(None)

# Remove linhas onde a coluna "Relatório da Partida" é "Confronto"
tabela = tabela[~tabela['Relatório da Partida'].str.contains("Confronto", na=False)]

# Adiciona a coluna de links na tabela
tabela['Link Relatório'] = links_relatorios[:len(tabela)]  # Ajusta o tamanho da lista para o tamanho da tabela

# Exibe a tabela final com os links
print("\nTabela com links:")


Tabela com links:


C:\Users\eduar\AppData\Local\Temp\ipykernel_32956\2417986439.py:36: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  a_tag = row.find('a', text='Relatório da Partida')


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://fbref.com/pt/comps/9/cronograma/Premier-League-Resultados-e-Calendarios"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Verifica a estrutura
    table_container = soup.find('div', {'class': 'table_container'})
    if table_container:
        rows = table_container.find_all('tr')
        tabela = extrair_tabela(rows)
        print(tabela)
    else:
        print("Tabela não encontrada. Verifique a estrutura HTML.")
else:
    print(f"Erro ao acessar a página: {response.status_code}")


Erro ao acessar a página: 429


In [5]:
# Desloca a coluna "Link Relatório" uma linha para cima
tabela['Link Relatório'] = tabela['Link Relatório'].shift(-1)

# Preenche a última linha da coluna "Link Relatório" com NaN (caso não esteja automaticamente)
tabela.loc[tabela.index[-1], 'Link Relatório'] = None


In [6]:
# Remove linhas onde "Link Relatório" é None
tabela = tabela[tabela['Link Relatório'].notna()]

In [7]:
tabela

,Sem,Dia,Data,Horário,Em casa,xG,Resultado,xG,Visitante,Público,Local,Árbitro,Relatório da Partida,Notas,Link Relatório
0,1,sex,2024-08-16,20:00,Manchester Utd,2.4,1–0,0.4,Fulham,73.297,Old Trafford,Robert Jones,Relatório da Partida,,/pt/partidas/cc5b4244/Manchester-United-Fulham...
1,1,sáb,2024-08-17,12:30,Ipswich Town,0.5,0–2,2.6,Liverpool,30.014,Portman Road Stadium,Tim Robinson,Relatório da Partida,,/pt/partidas/a1d0d529/Ipswich-Town-Liverpool-2...
2,1,sáb,2024-08-17,15:00,Newcastle Utd,0.3,1–0,1.8,Southampton,52.196,St James' Park,Craig Pawson,Relatório da Partida,,/pt/partidas/34557647/Newcastle-United-Southam...
3,1,sáb,2024-08-17,15:00,Nott'ham Forest,1.3,1–1,1.2,Bournemouth,29.763,The City Ground,Michael Oliver,Relatório da Partida,,/pt/partidas/4efc72e4/Nottingham-Forest-Bourne...
4,1,sáb,2024-08-17,15:00,Everton,0.5,0–3,1.4,Brighton,39.217,Goodison Park,Simon Hooper,Relatório da Partida,,/pt/partidas/71618ace/Everton-Brighton-and-Hov...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,12,sáb,2024-11-23,15:00,Everton,1.2,0–0,1.1,Brentford,38.915,Goodison Park,Chris Kavanagh,Relatório da Partida,,/pt/partidas/dd7675a7/Everton-Brentford-2024No...
126,12,sáb,2024-11-23,15:00,Fulham,1.0,1–4,1.3,Wolves,,Craven Cottage,Robert Jones,Relatório da Partida,,/pt/partidas/ed24aeb8/Fulham-Wolverhampton-Wan...
127,12,sáb,2024-11-23,17:30,Manchester City,2.1,0–4,2.5,Tottenham,,Etihad Stadium,John Brooks,Relatório da Partida,,/pt/partidas/c48b896c/Manchester-City-Tottenha...
128,12,dom,2024-11-24,14:00,Southampton,1.3,2–3,3.1,Liverpool,31.278,St Mary's Stadium,Samuel Barrott,Relatório da Partida,,/pt/partidas/eb6f8e39/Southampton-Liverpool-20...


In [8]:
tabela.to_csv(r'C:\Users\eduar\OneDrive\Desktop\DEV\Football\PremierLeague\links.csv', index= False)